# Processamento dos dados gerados

In [ ]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sistema_antifraude import config
from sistema_antifraude.utilidades.geolocalizacao import obter_geolocalizacao_do_ip
from sistema_antifraude.utilidades.geolocalizacao import obter_paises_estados_cidades_das_latlongs
from sistema_antifraude.utilidades.geolocalizacao import distancia_entre_latlongs_km

In [ ]:
df = pd.read_csv(os.path.join(config.DATA_PATH, 'raw', 'dados_gerados.csv'))

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [ ]:
df.head()

### Diferença de tempo entre acessos

In [ ]:
df['distancia_segundos_entre_acessos'] = 0
for user_id in df['user_id'].unique():
    registros_usuario = df[df['user_id'] == user_id].copy()
    registros_usuario.sort_values(by='timestamp', inplace = True)
    df.loc[
        df['user_id'] == user_id, 'distancia_segundos_entre_acessos'
    ] = (registros_usuario['timestamp'].diff().dt.total_seconds()).fillna(0).map(int)
df.head()

### Horário suspeito

In [ ]:
df['horario_suspeito'] = df['timestamp'].map(lambda timestamp: 1 if timestamp.hour < 6 else 0)
df['horario_suspeito'].value_counts(normalize = True)

### Dispositivo utilizado anteriormente pelo próprio usuário

In [ ]:
df['is_dispositivo_anterior_usuario'] = 0
for user_id in df['user_id'].unique():
    todos_acessos = df[df['user_id'] == user_id].copy()
    todos_acessos.sort_values(by='timestamp', inplace = True)
    for idx, acesso_atual in todos_acessos.iterrows():
        dispositivos_anteriores = todos_acessos[todos_acessos['timestamp'] < acesso_atual['timestamp']]['device_id'].values
        if acesso_atual['device_id'] in dispositivos_anteriores:
            df.loc[idx, 'is_dispositivo_anterior_usuario'] = 1
df['is_dispositivo_anterior_usuario'].value_counts(normalize = True)

### País e cidade através do IP

In [ ]:
df['cidade_ip'] = df['ipv4'].map(lambda ip: obter_geolocalizacao_do_ip(ip)['cidade'])
df['pais_ip'] = df['ipv4'].map(lambda ip: obter_geolocalizacao_do_ip(ip)['pais'])


In [ ]:
df['pais_ip'].value_counts()

### País e cidade através da lat-long

In [ ]:
geolocalizacoes = list(
    zip(
        df['latitude'].copy().astype(float).values, 
        df['longitude'].copy().astype(float).values
    )
)
df['pais_latlong'], df['estado_latlong'], df['cidade_latlong'] = \
    obter_paises_estados_cidades_das_latlongs(geolocalizacoes)
print(df['pais_latlong'].value_counts(normalize = True))

In [ ]:
df['pais_ip_latlong_concordancia'] = df['pais_ip'] == df['pais_latlong']
df['pais_ip_latlong_concordancia'] = df['pais_ip_latlong_concordancia'].map(lambda x: 1 if x else 0)
df['pais_ip_latlong_concordancia'].value_counts(normalize = True)

### Distância entre as localizações anteriores e atual em quilômetros

In [ ]:
df['distancia_latlong_anterior'] = 0.0
for user_id in df['user_id'].unique():
    registros_usuario = df[df['user_id'] == user_id].copy()
    registros_usuario.sort_values(by='timestamp', inplace = True)
    for idx, acesso_atual in registros_usuario.iterrows():
        latlong_anteriores = registros_usuario[registros_usuario['timestamp'] < acesso_atual['timestamp']][['latitude', 'longitude']].values
        if len(latlong_anteriores) > 0:
            latlong_anterior = latlong_anteriores[-1]
            distancia = distancia_entre_latlongs_km(
                (acesso_atual['latitude'], acesso_atual['longitude']), 
                (latlong_anterior[0], latlong_anterior[1])
            )
            df.loc[idx, 'distancia_latlong_anterior'] = round(distancia, 1)

In [ ]:
df.to_csv(os.path.join(config.DATA_PATH, 'processed', 'dados_processados.csv'), index = False)